# DataFrames Python Lab

**Remember**: In the notebook, you already have a `SQLContext` object, called `sqlContext`. However, if you need to create one yourself (e.g., for a non-notebook application), do it like this:
```
# assuming "sc" is some existing SparkContext object
sqlContext = SQLContext(sc)
```

**WARNING: Don't ever do this IN the notebook!**

We've already created a Parquet table containing popular first names by gender and year, for all years between 1880 and 2014. (This data comes from the United States Social Security Administration.) We can create a DataFrame from that data, by calling `sqlContext.read.parquet()`.

**NOTE**: That's the Spark 1.4 API. The API is slightly different in Spark 1.3.

In [4]:
# Spark 1.4
df = sqlContext.read.parquet("dbfs:/mnt/training/ssn/names.parquet")

# Spark 1.3
# df = sqlContext.parquetFile("dbfs:/mnt/training/ssn/names.parquet")

Let's cache our Social Security names DataFrame, to speed things up.

In [6]:
df.cache()

Let's take a quick look at the first 20 items of the data.

In [8]:
df.show()

You can also use the `display()` helper, to get more useful (and graphable) output:

In [10]:
display(df)

Take a look at the data schema, as well. Note that, in this case, the schema was read from the columns (and types) in the Parquet table.

In [12]:
df.printSchema()

You can create a new DataFrame that looks at a subset of the columns in the first DataFrame.

In [14]:
firstNamesDF = df.select("firstName", "year")

Then, you can examine the values in the `nameDF` DataFrame, using an action like `show()` or the `display()` helper:

In [16]:
display(firstNamesDF)

You can also count the number of items in the data set...

In [18]:
firstNamesDF.count()

...or determine how many distinct names there are.

In [20]:
firstNamesDF.select("firstName").distinct().count()

Let's do something a little more complicated. Let's use the original data frame to find the five most popular names for girls born in 1980. Note the `desc()` in the `orderBy()` call. `orderBy()` (which can also be invoked as `sort()`) sorts in ascending order. `desc()` causes the sort to be in _descending_ order for the column to which `desc()` is attached.

In [22]:
display(df.filter(df['year'] == 1980).
           filter(df['gender'] == 'F').
           orderBy(df['total'].desc(), df['firstName']).
           select("firstName").
           limit(5))

Note that we can do the same thing using the lower-level RDD operations. However, use the DataFrame operations, when possible. In general, they're more convenient. More important, though, they allow Spark to build a query plan that can be optimized through [Catalyst](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html).

## Joins

Let's use two views of our SSN data to answer this question: How popular were the top 10 female names of 2010 back in 1930?

Before we can do that, though, we need to define a utility function. Spark SQL doesn't support the SQL `LOWER` function. To ensure that our data matches up properly, it'd be nice to force the names to lower case before doing the match. Fortunately, it's easy to define our own `LOWER` function:

In [26]:
lower = udf(lambda s: s.lower())

In [27]:
from pyspark.sql.functions import *

In [28]:
lower

Okay, now we can go to work.

In [30]:
# Create a new DataFrame from the SSNA DataFrame, so that:
# - We have a lower case version of the name, for joining
# - We've weeded out the year
#
# NOTE: The aliases are necessary; otherwise, the query analyzer
# generates false equivalences between the columns.
ssn2010 = df.filter(df['year'] == 2010).\
             select(df['total'].alias("total2010"), 
                    df['gender'].alias("gender2010"), 
                    df['firstName'].alias("firstName2010"),
                    lower(df['firstName']).alias("name2010"))

# Let's do the same for 1930.
ssn1930 = df.filter(df['year'] == 1930).\
             select(df['total'].alias("total1930"), 
                    df['gender'].alias("gender1930"), 
                    df['firstName'].alias("firstName1930"),
                    lower(df['firstName']).alias("name1930"))

# Now, let's find out how popular the top 10 New York 2010 girls' names were in 1880.
# This join works fine in Scala, but not (yet) in Python.
#
# j1 = ssn2010.join(ssn1930, (ssn2010.name2010 == ssn1930.name1930) and (ssn2010.gender2010 == ssn1930.gender1930))

# So, we'll do a slightly different version.
joined = ssn2010.filter(ssn2010.gender2010 == "F").\
                 join(ssn1930.filter(ssn1930.gender1930 == "F"), ssn2010.name2010 == ssn1930.name1930).\
                 orderBy(ssn2010.total2010.desc()).\
                 limit(10).\
                 select(ssn2010.firstName2010.alias("name"), ssn1930.total1930, ssn2010.total2010)

display(joined)

## Assignment

In the cell below, you'll see an empty function, `top_female_names_for_year`. It takes three arguments:

* A year
* A number, _n_
* A starting DataFrame (which will be `df`, from above)

It returns a new DataFrame that can be used to retrieve the top _n_ female names for that year (i.e., the _n_ names with the highest _total_ values). If there are multiple names with the same total, order those names alphabetically.

Write that function. To test it, run the cell _following_ the function?i.e., the one containing the `run_tests()` function. (This might take a few minutes.)

In [33]:
def top_female_names_for_year(year, n, df):
  return df.limit(10) # THIS IS NOT CORRECT! FIX IT.

In [34]:
# Transparent Tests
from test_helper import Test
def test_year(year, df):
    return [row.firstName for row in top_female_names_for_year(year, 5, df).collect()]

In [35]:
def run_tests():
  Test.assertEquals(test_year(1945, df), [u'Mary', u'Linda', u'Barbara', u'Patricia', u'Carol'], 'incorrect top 5 names for 1945')
  Test.assertEquals(test_year(1970, df), [u'Jennifer', u'Lisa', u'Kimberly', u'Michelle', u'Amy'], 'incorrect top 5 names for 1970')
  Test.assertEquals(test_year(1987, df), [u'Jessica', u'Ashley', u'Amanda', u'Jennifer', u'Sarah'], 'incorrect top 5 names for 1987')
  Test.assertTrue(len(test_year(1945, df)) <= 5, 'list not limited to 5 names')
  Test.assertTrue(u'James' not in test_year(1945, df), 'male names not filtered')
  Test.assertTrue(test_year(1945, df) != [u'Linda', u'Linda', u'Linda', u'Linda', u'Mary'], 'year not filtered')
  Test.assertEqualsHashed(test_year(1880, df), "2038e2c0bb0b741797a47837c0f94dbf24123447", "incorrect top 5 names for 1880")
  
run_tests()

## Solution

If you're stuck, and you're really not sure how to proceed, feel free to check out the solution. You'll find it in the same folder as the lab.